Import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score , mean_squared_error,make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import random
import plotly.graph_objects as go
import plotly.express as px
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import tensorflow as tf

In [ ]:
diamond = pd.read_csv(os.path.join(dirname, filename),index_col = 'Unnamed: 0')

In [ ]:
diamond.head(5)

In [ ]:
diamond.shape

Check if there are missing values

In [ ]:
# Missing values
diamond.isna().sum()

In [ ]:
diamond.duplicated().sum()

In [ ]:
diamond = diamond.loc[~diamond.duplicated(),:]

In [ ]:
diamond.info()

In [ ]:
diamond.describe()

 we note that there are some rows with x,y,z = 0 
 it's technically not possible we display and drop this row

In [ ]:
zero_values = (diamond.loc[:,'x'] == 0) | (diamond.loc[:,'y'] == 0)| (diamond.loc[:,'z'] == 0)
zero_values.sum()

In [ ]:
diamond = diamond.loc[~zero_values]

In [ ]:

fig = px.histogram(diamond, x="price",
                   hover_data=diamond.columns,)
fig.show()

In [ ]:
fig = px.histogram(diamond, x="carat",
                   hover_data=diamond.columns)
fig.show()

In [ ]:
plt.figure(figsize = (20,6))


sns.factorplot(x='color', data=diamond , kind='count',aspect=3)



plt.show()

In [ ]:
plt.figure(figsize = (12,6))

sns.factorplot(x='cut', data=diamond , kind='count',aspect=3)
 
plt.show()

In [ ]:
plt.figure(figsize = (12,6))

sns.factorplot(x='clarity', data=diamond , kind='count',aspect=3)
 
plt.show()

In [ ]:
lst_features = list(diamond.columns)
drop_f = ['price','cut','color','clarity']

for index in drop_f:
  lst_features.remove(index)

fig = go.Figure()

for feature in lst_features:

   fig.add_trace(go.Box(y=diamond.loc[:,feature],name = feature))


fig.show()

Diamonds with color D are most common for character sizes <1.3
D colors are rare so it is normal to find only a few stone with this color for large stones
we can see that the price depends on the weight of the stone (carat)
Larger diamonds> 2 carat are usually medium H, J colors

In [ ]:
fig = px.scatter(diamond, x="carat", y="price",color = 'color')
fig.show()


we can see that the cheapest diamonds are those with a low clarity (I1)
we see that some diamonds with very high clarity (IF) even being light 1 carat are worth in the 20k dollars
high quality diamonds> 2.5 carats are diamonds with low clarity

In [ ]:
fig = px.scatter(diamond, x="carat", y="price",color = 'clarity')
fig.show()

In [ ]:
fig = px.box(diamond, x="cut", y="price",color = 'color')
fig.show()

In [ ]:
plt.figure(figsize = (12,6))

sns.heatmap(diamond.corr(),annot = True,linewidths=3,linecolor='black',cbar = False)

plt.show()

In [ ]:
diamond = pd.get_dummies(data = diamond,drop_first = True)

In [ ]:
diamond.head()

Scaling of data , We fit_transform the train and just transform the test to avoid data linkage

In [ ]:
X = diamond.drop('price',axis = 1)
y = diamond.loc[:,'price']
X_train,X_test, y_train ,y_test = train_test_split(X,y,test_size = .2, random_state = 42)

scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train.loc[:,['carat','depth','table','x','y','z']]),columns = ['carat','depth','table','x','y','z'],index = X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test.loc[:,['carat','depth','table','x','y','z']]),columns = ['carat','depth','table','x','y','z'], index = X_test.index)

X_train_scale_final = X_train.copy()
X_test_scale_final = X_test.copy()

X_train_scale_final.loc[:,['carat','depth','table','x','y','z']] = X_train_scaled.loc[:,['carat','depth','table','x','y','z']]
X_test_scale_final.loc[:,['carat','depth','table','x','y','z']] = X_test_scaled.loc[:,['carat','depth','table','x','y','z']]

Linear Regression

In [ ]:
lr = LinearRegression()
lr.fit(X_train_scale_final,y_train)
y_pred = lr.predict(X_test_scale_final)

mse = (mean_squared_error(y_test,y_pred)) 

rmse = mse**.5
rmse

RMSE means that on average we are wrong by 1128 dollars

In [ ]:
model = Sequential()

model.add(Dense(256,activation='relu',input_dim= X_train_scale_final.shape[1]))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='linear'))


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss = 'mse')

In [ ]:
model.fit(x = X_train_scale_final,y = y_train,validation_data=(X_test_scale_final,y_test),epochs = 200,batch_size=512)

In [ ]:
y_pred = model.predict(X_test_scale_final)

In [ ]:
mean_squared_error(y_test,y_pred) ** 0.5

RMSE means that on average we are wrong by 534 dollars so it's a good performance 